In [3]:
import jax
import flax
from flax import nnx
import orbax.checkpoint as ocp
import pandas as pd
import matplotlib.pyplot as plt

print(jax.__version__)
print(flax.__version__)
print(ocp.__version__)

path = ocp.test_utils.erase_and_create_empty("/tmp/repro-crash-model/")
options = ocp.CheckpointManagerOptions(max_to_keep=2)
mngr = ocp.CheckpointManager(path, options=options)


def save_model(epoch, model):
    state = nnx.state(model)
    mngr.save(epoch, args=ocp.args.StandardSave(state))
    mngr.wait_until_finished()

class TestModel(nnx.Module):
    def __init__(self, n1, n2, n3, *, rngs = nnx.Rngs(0)):
        self.m1 = nnx.Linear(n1, n2, rngs=rngs)
        self.dropout = nnx.Dropout(0.1, rngs=rngs)
        self.m1 = nnx.Linear(n2, n3, rngs=rngs)

    def __call__(self, x):
        x = self.m1(x)
        x = self.dropout(x)
        x = self.m2(x)
        return x

model = TestModel(2,3,4)

save_model(0, model)

0.7.2
0.12.0
0.11.25


In [5]:
state = nnx.pure(nnx.state(model, nnx.Param))

In [11]:
dict(nnx.flatten(state)[1])

{('m1', 'bias'): Array([0., 0., 0., 0.], dtype=float32),
 ('m1',
  'kernel'): Array([[-0.3872593 ,  0.3890168 ,  0.98396   , -0.5007451 ],
        [-1.2219061 ,  0.5536564 ,  0.87500995, -0.37101308],
        [ 0.7365868 ,  0.881758  , -1.0125103 , -0.2935628 ]],      dtype=float32)}